In [115]:
import zipfile
import pandas as pd

#accessing the zip file downloaded using Kaggle API
file_path = "/Users/papayaw/projects/Walmart Retail Pipeline/walmart-dataset.zip"
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall("walmart_data")

In [116]:
import pandas as pd
import os

 
def extract(store_data):
    """
    The goal of this function is to extract the dataset from the file.
    
    :param store_data: the file path where the grocery sales data is stored.
    :return: a dataset containing sales data.
    """
    store_df = pd.read_csv(store_data)

    return store_df

# Call the extract() function and store it as the "sales_df" variable
sales_df = extract("walmart_data/walmart.csv")

In [117]:
import numpy as np
import datetime

def transform(raw_data):
    """
    To prepare the data for analysis, a number of transformations 
    will be applied:
        -filling missing values of columns 'Weekly_Sales', 'CPI', and 'Unemployment' with their mean.
        -transform the 'Date' column to the datetime dtype.
        - dropping null values in the 'Date' column.
        -creating the 'Month' column using data in the 'Date' column.
        -creating the 'Year' column using data in the 'Date' column.
        - selecting rows where 'Weekly_Sales' is greater than 10,000.
        - selecting the desired columns for analysis.
    
    :param raw_data: the sales dataframe.
    :return: a cleaned dataset having had the operations above applied to it
    """

    #filling missing values with the mean
    mean_weekly_sales = np.mean(raw_data['Weekly_Sales'])
    mean_CPI = np.mean(raw_data['CPI'])
    mean_unemployment = np.mean(raw_data['Unemployment'])
    #applying mean
    raw_data['Weekly_Sales'] = raw_data['Weekly_Sales'].fillna(mean_weekly_sales)
    raw_data['CPI'] = raw_data['CPI'].fillna(mean_CPI)
    raw_data['Unemployment'] = raw_data['Unemployment'].fillna(mean_unemployment)
    #changing 'Date' dtype to datetime
    raw_data['Date'] = pd.to_datetime(raw_data['Date'],format='%d-%m-%Y')
    #dropping null values for the date column
    raw_data['Date'] = raw_data['Date'].dropna()
    #adding month column
    raw_data['Month'] = raw_data['Date'].dt.month
    #adding the year column
    raw_data['Year'] = raw_data['Date'].dt.year
    #keeping rows where weekly_sales>10000
    raw_data = raw_data[raw_data['Weekly_Sales']>10000]
    #selecting the relevant columns 
    relevant_columns = ['Store','Month', 'Year','Holiday_Flag','Weekly_Sales','CPI','Unemployment']
    clean_df = raw_data.loc[:,relevant_columns]

    return clean_df


In [118]:
# Call the transform() function and pass the merged DataFrame
clean_data = transform(sales_df)

In [119]:
# Create the avg_weekly_sales_per_month function that takes in the cleaned data from the last step
def avg_weekly_sales_per_month(clean_data):
    
    # grouping month and weekly_sales columns
    group_df = clean_data.groupby('Month')
    new_data = group_df['Weekly_Sales'].agg('mean').round(2)
    new_data = new_data.reset_index()
    new_data = new_data.rename(columns={'Weekly_Sales':'Avg_Sales'})
    

    return new_data


In [120]:
# Call the avg_weekly_sales_per_month() function and pass the cleaned DataFrame
agg_data = avg_weekly_sales_per_month(clean_data)

In [121]:
# Create the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    full_data.to_csv('clean_data.csv', index=False)
    agg_data.to_csv('agg_data.csv', index=False)


In [122]:
# Call the load() function and pass the cleaned and aggregated DataFrames with their paths    
load(clean_data, 'clean_data.csv', agg_data, 'agg_data.csv')

In [123]:
# Create the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
import os

def validation(file_path):
    if os.path.exists(file_path):
        print('Exists')
    else:
        raise Exception(f"File not found at path {file_path}")


In [124]:
# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
validation('clean_data.csv')
validation('agg_data.csv')

Exists
Exists
